In [3]:
import requests
from bs4 import BeautifulSoup

import time
import random
import logging

In [4]:
logger = logging.getLogger(__name__)

In [5]:
BASIC_FORMAT = "%(asctime)s-%(levelname)s-%(message)s"
chlr = logging.StreamHandler()
chlr.setFormatter(logging.Formatter(BASIC_FORMAT))
logger.setLevel('DEBUG')
logger.addHandler(chlr)

In [87]:
class RentalCrawler:
    DOMAIN_URL = 'https://rent.591.com.tw/'
    GET_LIST_URL = 'https://rent.591.com.tw/home/search/rsList'
    HEADERS = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }

    def __init__(self):
        self.session = requests.Session()

    def _get_token(self):
        res = self.session.get(self.DOMAIN_URL, headers=self.HEADERS)
        soup = BeautifulSoup(res.text, 'html.parser')
        token = soup.select_one('meta[name="csrf-token"]').get('content')
        return token

    def _get_rental_list(self, token: str, region: int=3, first_row: int=0):
        headers = self.HEADERS.copy()
        headers['X-CSRF-TOKEN'] = token
        params = f'is_format_data=1&is_new_list=1&type=1&region={region}&firstRow={first_row}'
        res = self.session.get(self.GET_LIST_URL, params=params, headers=headers)
        return res

In [88]:
crawler = RentalCrawler()

In [89]:
token = crawler._get_token()

In [90]:
res = crawler._get_rental_list(token=token, region=3, first_row=0)

In [91]:
res.json()['records']

'10,999'

In [70]:
res.json()['data']['data']

[{'title': '捷運一分鐘SOGO東區摩登優質套房',
  'type': '1',
  'post_id': 12699348,
  'kind_name': '分租套房',
  'room_str': '',
  'floor_str': '4F/4F',
  'community': '',
  'price': '16,800',
  'price_unit': '元/月',
  'photo_list': ['https://img2.591.com.tw/house/2022/05/02/165146505623750405.jpg!510x400.jpg',
   'https://img1.591.com.tw/house/2022/05/02/165146505623710908.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2021/07/28/162745585506894609.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2021/07/20/162676654472473106.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2021/07/20/162676654243873102.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2021/07/28/162745585525397502.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2021/07/20/162676660377755203.jpg!510x400.jpg',
   'https://img1.591.com.tw/house/2019/09/20/156896830030617105.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2019/09/20/156896830089823900.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2021/09/23/16324

In [100]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}
s = requests.Session()
url = 'https://rent.591.com.tw/'
r = s.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
token_item = soup.select_one('meta[name="csrf-token"]')

In [101]:
headers = headers.copy()
headers['X-CSRF-TOKEN'] = token_item.get('content')

In [105]:
j = requests.jar()

AttributeError: module 'requests' has no attribute 'jar'

In [ ]:
requests.cookies

In [109]:
url = 'https://rent.591.com.tw/home/search/rsList'
# params = 'is_format_data=1&is_new_list=1&type=1&region=3&firstRow=0'
params = 'is_format_data=1&is_new_list=1&type=1&region=3'
r = s.get(url, params=params, headers=headers, cookies={'urlJumpIp': '1'})

In [84]:
# https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3
r = s.get('', params=params, headers=headers)

In [110]:
r.json()['data']['data']

[{'title': '大安路敦化南路名人巷雅房',
  'type': '1',
  'post_id': 12543664,
  'kind_name': '雅房',
  'room_str': '',
  'floor_str': '4F/4F',
  'community': '',
  'price': '8,700',
  'price_unit': '元/月',
  'photo_list': ['https://img1.591.com.tw/house/2020/09/08/159956449012613958.jpg!510x400.jpg',
   'https://img1.591.com.tw/house/2020/09/08/159956449012799890.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2020/09/08/159956449012790923.jpg!510x400.jpg',
   'https://img1.591.com.tw/house/2020/09/08/159956449012774160.jpg!510x400.jpg',
   'https://img1.591.com.tw/house/2020/09/08/159956449012704803.jpg!510x400.jpg',
   'https://img1.591.com.tw/house/2020/09/08/159956449012784609.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2020/09/08/159956449012817448.jpg!510x400.jpg',
   'https://img1.591.com.tw/house/2020/09/08/159956449012732725.jpg!510x400.jpg',
   'https://img1.591.com.tw/house/2020/09/08/159956449012822781.jpg!510x400.jpg',
   'https://img1.591.com.tw/house/2020/09/08/1599564490129

In [9]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}
s = requests.Session()
url = f'https://rent.591.com.tw/home/12670629'
r = s.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
token_item = soup.select_one('meta[name="csrf-token"]')

headers = headers.copy()
headers['X-CSRF-TOKEN'] = token_item.get('content')
headers['deviceid'] = s.cookies.get_dict()['T591_TOKEN']
# headers['token'] = s.cookies.get_dict()['PHPSESSID']
headers['device'] = 'pc'

url = f'https://bff.591.com.tw/v1/house/rent/detail?id=12670629'
r = s.get(url, headers=headers)

In [11]:
r.json()['data']

{'breadcrumb': [{'name': '台北市',
   'id': 1,
   'query': 'region',
   'link': '/?region=1'},
  {'name': '萬華區', 'id': 6, 'query': 'section', 'link': '/?region=1&section=6'},
  {'name': '分租套房',
   'id': 3,
   'query': 'kind',
   'link': '/?region=1&section=6&kind=3'}],
 'title': '西門新套房可炊有烘衣機廚房',
 'deposit': '押金二個月',
 'kind': 3,
 'relieved': 0,
 'regionId': 1,
 'sectionId': 6,
 'shareInfo': {'url': 'https://www.591.com.tw/1R?salt=BYsK&s=',
  'from': '來自【591租屋】',
  'title': '台北市萬華區，分租套房出租，12867元/月，詳情：'},
 'dealText': '',
 'dealTime': 0,
 'browse': {'pc': 70, 'mobile': 161},
 'tags': [{'id': 2, 'value': '近捷運'},
  {'id': 3, 'value': '拎包入住'},
  {'id': 4, 'value': '近商圈'},
  {'id': 5, 'value': '隨時可遷入'},
  {'id': 6, 'value': '可開伙'}],
 'price': '12,867',
 'priceUnit': '元/月',
 'navData': [{'title': '位置與周邊', 'key': 'positionRound', 'active': 1},
  {'title': '設備與服務', 'key': 'service', 'active': 1},
  {'title': '屋況介紹', 'key': 'remark', 'active': 1},
  {'title': '房屋詳情', 'key': 'houseDetail', 'active': 